<a href="https://colab.research.google.com/github/gikirima/indonews-scrapper/blob/main/Bersihkan_isi_berita_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [ ]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 3.9 MB/s eta 0:00:00
Mounted at /content/drive


# Generated Code

In [ ]:
# To run this code you need to install the following dependencies:
# pip install google-genai tqdm

import base64
import os
import pandas as pd
from google import genai
from google.genai import types
from tqdm.notebook import tqdm # Import tqdm


def generate(isi_berita):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-flash-lite-preview-06-17"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=f"""Berikut teks yang perlu dibersihkan:
\"\"\"
{isi_berita}
\"\"\"
"""),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        thinking_config = types.ThinkingConfig(
            thinking_budget=0,
        ),
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text="""
            Anda adalah AI penyunting teks berita. Tugas Anda adalah mengubah artikel mentah menjadi teks berita yang bersih dan profesional. Ikuti semua aturan di bawah ini secara ketat.

            **Urutan Penyuntingan:**

            **A. Bagian Awal & Judul Teks**
            1.  **HAPUS DATELINE:** Hapus baris pembuka yang berisi nama media dan lokasi (misal: "KOMPAS.com, Jakarta - ", "TRIBUNNEWS.COM - ").

            **B. Bagian Isi Teks**
            2.  **HAPUS IKLAN & PROMOSI:** Hapus semua bentuk iklan (misal: "Advertisement", "Scroll untuk melanjutkan") dan promosi platform lain (misal: "Download aplikasi...", "Saksikan di YouTube...").
            3.  **HAPUS REFERENSI INTERNAL:** Hapus frasa rujukan artikel lain (misal: "Baca Juga:", "Simak Juga:", dan variasinya) beserta kalimat atau tautan yang mengikutinya.
            4.  **HAPUS TAG MEDIA:** Hapus semua placeholder media (misal: "[Gambas:Video 20detik]", "[FOTO]").
            5.  **PERBAIKI TEKS:**
                * **Kesalahan Tik (Typo):** Koreksi ejaan kata yang salah (misal: `pmerintah` → `pemerintah`). Gunakan konteks untuk akurasi.
                * **Spasi:** Perbaiki spasi yang hilang (misal: `LIVEmalam` → `LIVE malam`) dan hapus spasi ganda.
                * **Singkatan:** JANGAN ubah singkatan umum (misal: "yg", "utk", "dll", "pemilu").

            **C. Bagian Akhir Teks**
            6.  **HAPUS AJAKAN INTERAKTIF:** Hapus kalimat ajakan (misal: "Beri komentar di bawah.", "Bagikan artikel ini.").
            7.  **HAPUS DISCLAIMER:** Hapus seluruh kalimat atau paragraf disclaimer.
            8.  **HAPUS KREDIT:** Hapus baris kredit di akhir teks (misal: "Penulis:", "Editor:", "Sumber:").

            **D. Aturan Finalisasi & Output**
            9.  **STANDARKAN FORMAT:** Ganti semua tanda petik (`“...“`, `‘...’`) menjadi tanda petik lurus (`"..."`).
            10. **PERTAHANKAN INTI BERITA:** Jaga keutuhan semua fakta, data, dan kutipan narasumber.

            **PERINTAH OUTPUT FINAL:**
            Kirimkan **HANYA** teks berita hasil akhir yang sudah bersih. Jangan menyertakan komentar, penjelasan, atau format tambahan apa pun.
            """),
        ],
    )

    cleaned_text = ""
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if chunk.text is not None:
          cleaned_text += chunk.text
    return cleaned_text


if __name__ == "__main__":
    # Load your dataframe here. Replace 'your_dataset.csv' with your actual file path.
    # For example:
    df = pd.read_csv('https://raw.githubusercontent.com/gikirima/indonews-scrapper/refs/heads/main/Final%20Dataset%20Clean/old_dataset_clean.csv')

    # Apply the generate function to the 'isi_berita' column and create a new column
    # using tqdm for a loading bar
    tqdm.pandas() # Initialize pandas integration with tqdm
    df['isi_berita_bersih'] = df['isi'].progress_apply(generate)

    # Display the updated DataFrame
    display(df)

    # Save the cleaned data to a new CSV file
    df.to_csv('/content/drive/MyDrive/Google AI Studio/dataset_old_cleaned.csv')

  0%|          | 0/39476 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# cek NaN
df.isnull().sum()

,0
sumber,0
url,0
judul,9
author,5
tanggal_publikasi,0
isi_berita,0
jumlah_token_chatgpt,0
isi_berita_bersih,0
